# NMMA TUTORIAL 

## Generating Light Curves
1. Create an injection file

The general way to create an injection file is running the command ```nmma-create-injection```. It takes a prior file, which specifies the parameter space that is sampled to create injections. An example is shown below:

```nmma-create-injection -f lc_dir/injection --prior-file priors/Ka2017.prior --eos-file example_files/eos/ALF2.dat --binary-type BNS --n-injection 100 --original-parameters --extension json```

This will create a json file containing 100 BNS injections sampled from the Me2017.prior file. We can use this injection file to generate light curves. 

2. Create light curves

The command ```lightcurve-generation``` takes an injection file and other specified parameters and generates a file containing the simulated observations. ``--dt`` determines the cadence of the observations. 

```lightcurve-generation --model Ka2017 --outdir lc_dir --label test --tmin -2 --tmax 20 --dt 0.25 --filters ztfg,ztfr,ztfi --outfile-type json --injection lc_dir/injection.json --injection-detection-limit 22.0,22.0,22.0 --ztf-uncertainties --increment-seeds```

3. Run Bayesian inference

We can run a parameter estimation analysis through the command ```light_curve_analysis```. This takes a light curve and a prior that constrains the parameter space which the sampler searches over. By default, the sampler used is ```PyMultiNest```, but this can be changed. An example analysis can be run as follows:

```light_curve_analysis --model Ka2017 --outdir lan_inf --label test_0 --trigger-time 44240 --data lc_dir/test_0.json --prior priors/Ka2017.prior --injection-outfile lan_inf/lcanalysis.csv --filters ztfg,ztfr,ztfi``` 


# HOW TO USE FOR LFI

1. Injection files

To train our model, we rely on using two views of the data: a "fixed" and a "shifted/varied" light curve. To do this, we need two injection files that have repeats and contain the same injections (but have different noise instances). It is easiest to do this by first creating a varied injection file with a modified Ka2017.prior file. For example, I use a Ka2017_varied.prior file that has the time and luminosity distance varied. Note here that -n is the number of injections, -r is the number of repeats, and -s is the seed. 

```nmma-create-injection --prior-file priors/Ka2017_varied.prior -f lc_dir/injection_varied -e json -n 500 -r 50 -s 0 --eos-file example_files/eos/ALF2.dat --binary-type BNS --original-parameters```

To make the injection fixed, I use a python script (provided in the tutorial section in the LFI repo: https://github.com/malinadesai/Kilo/tree/main/tutorials). This script takes the varied injection and will change the time and the distance to a fixed value. Now, we have two injection files from which we can generate light curves from. 

2. Generate the light curves

Because the model generally requires a substantial amount of light curves to train, it is easiest to use a GPU allocation to speed up the process. This is possible by using salloc or slurm to either run commands on a GPU interactively or batch jobs to run later. A sample slurm script is available in the tutorial folder. 

3. Some notes

We want to ensure different noise instances for all light curves. This can be done by using the ```ztf-uncertainties``` tag when simulating light curves observed by ZTF. Additionally, changing the injection seed is important. Each new injection file to generate additional data should be created with a new seed to prevent duplicated injection parameters. This is also why the ```increment-seeds``` parameter should be used when running ```lightcurve-generation```. 